In [1]:
import json
import nltk
import inflect
import tqdm
import time
import requests 
from bs4 import BeautifulSoup as bs
import gensim
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import numpy as np
import collections
import json
import sys
sys.path.insert(0, "/Users/Lionel/Documents/Ada/Project/python")
import clean_recipes_datasets as clean
engine = inflect.engine()

In [2]:
# Load the recipes for which we have the quantity for each ingredients 
with open('../generated/1m_quant_usda_recipes.json') as json_data:
    quantities_recipes = json.load(json_data)

# Table of conversion for the unit found in 1M
with open('../generated/convert_gr.json') as json_data:
    convert_gr = json.load(json_data)
    
# Mapping from ingredient to USDA id
with open('../generated/???.json') as json_data:
    mapping_usda_id = json.load(json_data)
    
nutrients_mapping = pd.read_hdf("../generated/nut_data.h5", 'table')

In [6]:
# Get the profile of each recipes contained in 'recipes'
def map_recipes(recipes, mapping):
    threshold = 0.3
    profile = []
    mismatches_ratio_in_recipes = []

    for recipe in tqdm.tqdm(recipes):
        nb_ingr = len(recipe)
        mismatches = 0.0
        recipe_mapped = []
        
        for ingredient in recipe:
            if ingredient.startswith('usda_id'):
                usda_id = int(ingredient.split('=')[1])
                res_map = nutrients_mapping.loc[usda_id]
                recipe_mapped.append(res_map.tolist())
            else:
                mismatches += 1
                
        recipe_mapped = pd.DataFrame(recipe_mapped)
        mismatches_ratio_in_recipes.append(mismatches / nb_ingr)
        
        if mismatches / nb_ingr < threshold:
            recipe_mapped = recipe_mapped.sum(axis = 0)
            profile.append(recipe_mapped.tolist())
        else:
            profile.append(list())

    profile = pd.DataFrame(profile)
    profile.columns = mapping.columns
    
    return profile, mismatches_ratio_in_recipes

In [3]:
# We convert every quantities units to grams, and delete the recipes that contains unconvertable unit
to_remove = []
for recipe in quantities_recipes:
    for ingr in recipe:
        try:
            ingr[1] = convert_gr[ingr[1]]
            ingr[2] = clean.clean_ingredient(ingr[2])
        except KeyError:
            to_remove.append(recipe)
quantities_recipes = [list(map(lambda x: (x[0] * x[1], x[2]), recipe)) for recipe in quantities_recipes if recipe not in to_remove]

In [4]:
# Get median weight for each ingredient
ingredient_weight = []
for recipe in quantities_recipes:
    for ingredient in recipe:
        ingredient_weight.append(ingredient)
ingredient_weight = np.array(ingredient_weight)
ingredient_weight_df = pd.DataFrame(ingredient_weight, columns=['weight', 'ingredient'])
ingredient_weight_df = ingredient_weight_df.astype({'weight': float})
median_weight_by_ingredient = ingredient_weight_df.groupby('ingredient').median().reset_index()

In [7]:
quantities_recipes

[[(128.0, 'elbow macaroni'),
  (128.0, 'american cheese'),
  (64.0, 'celery'),
  (64.0, 'green pepper'),
  (42.900000000000006, 'pimento'),
  (64.0, 'mayonnaise'),
  (14.3, 'vinegar'),
  (3.1500000000000004, 'salt'),
  (2.1, 'dill weed')],
 [(1536.0, 'coconut'),
  (453.592, 'lean beef'),
  (21.450000000000003, 'garlic'),
  (2.5, 'kosher salt'),
  (14.3, 'lemon juice'),
  (14.3, 'soy sauce'),
  (28.6, 'cornstarch'),
  (224.0, 'pineapple chunk'),
  (448.0, 'mandarin orange'),
  (1536.0, 'cashew nut')],
 [(224.0, 'nonfat vanilla yogurt'),
  (128.0, 'strawberry'),
  (32.0, 'lowfat granola')],
 [(4.2, 'fennel seed'),
  (453.592, 'pork tenderloin'),
  (235, 'fennel bulb'),
  (28.6, 'olive oil'),
  (3, 'garlic clove'),
  (32.0, 'white wine'),
  (64.0, 'reducedsodium chicken broth'),
  (28.6, 'butter'),
  (2.1, 'lemon juice')],
 [(200.0, 'cake flour'),
  (4.2, 'baking powder'),
  (20.0, 'sugar'),
  (7.0, 'white sesame seed'),
  (70.0, 'water'),
  (20.0, 'vegetable oil'),
  (80.0, 'sugar'),
  (

In [8]:
nutrients_mapping

nutr_per_100g                                                         \
nutrient       Alanine Calcium, Ca Carbohydrate, by difference Choline, total   
food_id                                                                         
1001             0.029        24.0                        0.06           18.8   
1002             0.029        23.0                        0.00           18.8   
1003             0.010         4.0                        0.00           22.3   
1004             0.644       528.0                        2.34           15.4   
1005             0.670       674.0                        2.79           15.4   
1006             0.859       184.0                        0.45           15.4   
1007             0.819       388.0                        0.46           15.4   
1008             0.711       673.0                        3.06            NaN   
1009             0.751       710.0                        3.37           16.5   
1010             0.659       643.0                        4.78            NaN   
1011             0.670       685.0                        2.57           15.4   
1012             0.384        83.0                        3.38           18.4   
1013             0.551        53.0                        4.61           17.5   
1014             0.357        86.0                        6.66           17.9   
1015             0.409       111.0                        4.76           16.3   
1016             0.643        61.0                        2.72           17.5   
1017             0.184        97.0                        5.52           26.8   
1018             0.764       731.0                        1.43           15.4   
1019             0.639       493.0                        3.88           15.4   
1020             0.823       550.0                        1.55           15.4   
1021             0.325       400.0                       42.65            NaN   
1022             0.762       700.0                        2.22           15.4   
1023             0.958      1011.0                        0.36           15.4   
1024             0.666       497.0                        0.49           15.4   
1025             0.691       746.0                        0.68           15.4   
1026             0.707       505.0                        2.40           15.4   
1027             0.660       575.0                        2.47           15.0   
1028             0.741       782.0                        2.77           15.4   
1029             0.757       697.0                        5.58           14.2   
1030             0.675       717.0                        1.12           15.4   
...                ...         ...                         ...            ...   
43114              NaN        55.0                       82.32          177.0   
43125              NaN        13.0                        2.80           13.5   
43128              NaN        35.0                        3.64           71.6   
43130              NaN        33.0                        7.70           84.2   
43132              NaN        25.0                        8.00           88.4   
43134              NaN        95.0                        9.00           82.0   
43135              NaN        44.0                        9.00            2.6   
43137              NaN        29.0                        8.00           73.1   
43212              NaN       101.0                       28.60          134.8   
43217            0.037        14.0                        5.31            9.9   
43273            0.623        56.0                        3.00           17.5   
43276              NaN        71.0                        3.50           27.2   
43278              NaN       562.0                       11.60            0.0   
43285              NaN        17.0                        7.50          255.7   
43299              NaN       188.0                        6.90           62.5   
43344              NaN         0.0      